<a href="https://colab.research.google.com/github/lattehauma/dacon/blob/main/%EC%B9%B4%EB%93%9C1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 신용카드 사기 거리 탐지 AI

* DACON 알고리즘 연습대회  

https://dacon.io/competitions/official/235930/codeshare/5236?page=1&dtype=recent

In [23]:
import pandas as pd
import numpy as np

In [3]:
train=pd.read_csv('/content/drive/MyDrive/colab/dacon/card/train.csv')
test=pd.read_csv('/content/drive/MyDrive/colab/dacon/card/test.csv')
val=pd.read_csv('/content/drive/MyDrive/colab/dacon/card/val.csv')

In [4]:
print(train.shape)
print(test.shape)
print(val.shape)

(113842, 31)
(142503, 31)
(28462, 32)


In [5]:
train.head()

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,3,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,4.983721,-0.994972
1,4,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,1.418291,-0.994972
2,6,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,-0.256131,-0.994960
3,8,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,0.262698,-0.994901
4,9,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,0.994900,-0.994901


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113842 entries, 0 to 113841
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      113842 non-null  int64  
 1   V1      113842 non-null  float64
 2   V2      113842 non-null  float64
 3   V3      113842 non-null  float64
 4   V4      113842 non-null  float64
 5   V5      113842 non-null  float64
 6   V6      113842 non-null  float64
 7   V7      113842 non-null  float64
 8   V8      113842 non-null  float64
 9   V9      113842 non-null  float64
 10  V10     113842 non-null  float64
 11  V11     113842 non-null  float64
 12  V12     113842 non-null  float64
 13  V13     113842 non-null  float64
 14  V14     113842 non-null  float64
 15  V15     113842 non-null  float64
 16  V16     113842 non-null  float64
 17  V17     113842 non-null  float64
 18  V18     113842 non-null  float64
 19  V19     113842 non-null  float64
 20  V20     113842 non-null  float64
 21  V21     11

null이 없는 완벽하게 정제된 데이터인듯

In [7]:
val.columns

Index(['ID', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30',
       'Class'],
      dtype='object')

In [12]:
val_cont=val['Class'].value_counts()[1]/val['Class'].value_counts()[0]

정상비율 대비 사기 거래 비율

### 모델 작성연습

In [10]:
train_x=train.drop(columns=['ID'])

In [11]:
len(train_x)

113842

In [13]:
from sklearn.ensemble import IsolationForest
model = IsolationForest(n_estimators=125, max_samples=len(train_x), contamination=val_cont, random_state=42)

In [14]:
model.fit(train_x)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


IsolationForest(contamination=0.0010551491277433877, max_samples=113842,
                n_estimators=125, random_state=42)

In [15]:
val_x=val.drop(columns=['ID','Class'])
val_y=val['Class']

val_pred=model.predict(val_x)

In [24]:
val_pred=np.where(val_pred==1, 0, val_pred)
val_pred=np.where(val_pred==-1, 1, val_pred)

In [31]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

val_score=f1_score(val_y, val_pred, average='macro')
print(classification_report(val_y, val_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.41      0.40      0.41        30

    accuracy                           1.00     28462
   macro avg       0.71      0.70      0.70     28462
weighted avg       1.00      1.00      1.00     28462



### 테스트 셋 돌리기

In [34]:
test.columns

Index(['ID', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30'],
      dtype='object')

In [35]:
test_x=test.drop(columns=['ID'])
test_pred=model.predict(test_x)
test_pred=np.where(test_pred==1, 0, test_pred)
test_pred=np.where(test_pred==-1, 1, test_pred)

In [36]:
submit=pd.read_csv('/content/drive/MyDrive/colab/dacon/card/sample_submission.csv')
submit.head()

,ID,Class
0,AAAA0x1,1
1,AAAA0x2,1
2,AAAA0x5,1
3,AAAA0x7,1
4,AAAA0xc,1


In [ ]:
submit['Class']=test_pred
submit.to_csv('/content/drive/MyDrive/colab/dacon/card/submit1.csv',index=False)